In [136]:
using Pkg, Gurobi, JuMP, CSV, DataFrames, LinearAlgebra, Statistics, Plots

## Formulation:

![alt text](Network_Diagram.jpg "Network Diagram")


In [137]:
chemicals = CSV.read("chemicals.csv", DataFrame)
p = size(chemicals)[2]
# print(chemicals)
p_matrix = Matrix(hcat(Vector(chemicals[2, 2:p]), Vector(chemicals[3, 2:p]), Vector(chemicals[5, 2:p]), Vector(chemicals[4, 2:p]), Vector(chemicals[1, 2:p]))')

5×4 Matrix{Float64}:
 128.98  0.00012898   449.0  0.45
  40.0   4.0e-5       492.0  0.49
  58.44  5.844e-5       0.0  0.0
  18.0   1.8e-5         0.0  0.0
  92.52  9.252e-5    1840.0  1.84

In [138]:
customers = CSV.read("customers.csv", DataFrame)
k_matrix = Matrix(customers)
# k_matrix[:, 1] = 1000*k_matrix[:, 1]
# k_matrix

6×3 Matrix{Float64}:
 1840.0  5000.0  0.15
 2000.0  3000.0  0.12
 4000.0  1000.0  0.05
 1500.0  8000.0  0.14
 1700.0  8000.0  0.17
 3000.0  5000.0  0.08

In [139]:
reactors = CSV.read("reactors.csv", DataFrame)
r_matrix = Matrix(reactors)

5×4 Matrix{Float64}:
 500000.0    100.0  10000.0  0.9
 800000.0    175.0  10000.0  0.95
      1.0e6  200.0  10000.0  0.99
 300000.0     80.0  10000.0  0.8
 400000.0     85.0  10000.0  0.88

In [140]:
separators_1 = CSV.read("separators1.csv", DataFrame)
m_matrix = Matrix(separators_1)

5×4 Matrix{Float64}:
 100000.0   50.0  10000.0  0.05
 200000.0  100.0  20000.0  0.01
 250000.0  120.0  21000.0  0.005
  50000.0   35.0  70000.0  0.2
  75000.0   45.0  80000.0  0.1

In [141]:
separators_2 = CSV.read("separators2.csv", DataFrame)
s_matrix = Matrix(separators_2)

5×4 Matrix{Float64}:
 100000.0  50.0  10000.0  0.9
 200000.0  50.0  10000.0  0.95
 250000.0  50.0  10000.0  0.99
  50000.0  50.0  10000.0  0.85
  75000.0  50.0  10000.0  0.88

In [142]:
suppliers_DCH = CSV.read("suppliers_DCH.csv", DataFrame)
n_p1_matrix = Matrix(suppliers_DCH)

5×3 Matrix{Float64}:
  450.0  10000.0  0.05
  500.0  10000.0  0.04
 1000.0   5000.0  0.005
  250.0  10000.0  0.2
  750.0   5000.0  0.02

In [143]:
suppliers_NAOH = CSV.read("suppliers_NAOH.csv", DataFrame)
n_p2_matrix = Matrix(suppliers_NAOH)

5×3 Matrix{Float64}:
  500.0  2500.0  0.05
  600.0  2000.0  0.04
  400.0  3000.0  0.07
 1000.0  2000.0  0.005
  800.0  2500.0  0.01

In [144]:
n_matrix = vcat(Matrix(suppliers_DCH), Matrix(suppliers_NAOH))

10×3 Matrix{Float64}:
  450.0  10000.0  0.05
  500.0  10000.0  0.04
 1000.0   5000.0  0.005
  250.0  10000.0  0.2
  750.0   5000.0  0.02
  500.0   2500.0  0.05
  600.0   2000.0  0.04
  400.0   3000.0  0.07
 1000.0   2000.0  0.005
  800.0   2500.0  0.01

In [145]:
wastes = CSV.read("wastes.csv", DataFrame)
waste_matrix = Matrix(wastes)
j = waste_matrix[1, 1]
l_aqueous = waste_matrix[1, 2]
l_organic = waste_matrix[1, 3]

500

\begin{align}
    \text{Sets of Values:}& \\
    \text{Supplier ($n$)}& \in [1, N] \\
    \text{Material ($p$)}& \in [1, P] \\
    \text{$p=1$ for DCH, $p=2$ for NaOH, $p=3$ for NaCl, $p=4$ for H2O, } & \text{and $p=5$ ECH} \\
    \text{Separator (Left) ($m$)}& \in [1, M] \\
    \text{Waste Water Treatment Plant (for initial separators) ($j$)}& \in [J] \\
    \text{Reactors ($r$)}& \in [1, R] \\
    \text{Separator (Right) ($s$)}& \in [1, S] \\
    \text{Customers ($k$)}& \in [1, K] \\
    \text{Waste Water Treatment Plant (for second separators) ($l$)}& \in [L] \\ \\
    \text{Fixed Cost for using separator (left) ($m$)} & = f_{m} \\
    \text{Fixed Cost for using reactor ($r$)} & = f_{r} \\
    \text{Fixed Cost for using separator (right) ($s$)} & = f_{s} \\ \\
    \text{Unit Cost for sending material to separator (left) ($m$)} & = c_{m} \\
    \text{Unit Cost for sending material to waste water treatment plant ($j$)} & = c_{j} \\
    \text{Unit Cost for sending material to reaction ($r$)} & = c_{r} \\
    \text{Unit Cost for sending material to separator (right) ($s$)} & = c_{s} \\
    \text{Unit Cost for sending material to waste water treatment plant (aqueous) ($l$)} & = c^{A}_{l} \\
    \text{Unit Cost for sending material to waste water treatment plant (organic) ($l$)} & = c^{O}_{l} \\
\end{align}

In [146]:
## Existing Data
# Materials: (1) DHC, (2) NaOH, (3) NaCl, (4) H2O, (5) ECH (material index)

# p_matrix: 5x4 matrix 
    # Rows: DHC, NaOH, NaCl, H2O, ECH (material index)
    # Columns: [1] MW (g/mol), [2] MW (MT/mol), [3] Price (USD/MT), [4] Price (USD/kg)
# n_p1_matrix: 5x3 matrix (DCH Suppliers)
    # Rows: DCH Supplier index
    # Columns: [1] Price ($/MT), [2] Supply, [3] Impurity %
# n_p2_matrix: 5x3 matrix (NaOH Suppliers)
    # Rows: NaOH Supplier index
    # Columns: [1] Price ($/MT), [2] Supply, [3] Impurity %
# m_matrix: 5x4 matrix [Left Separators]
    # Rows: Separator (Left) index
    # Columns: [1] Fixed Cost, [2] Flow Cost, [3] Capacity, [4] Impurity %
# waste_matrix: 1x3 matrix (all waste water treatment plants)
    # j: Int64 (Unit Cost of waste water treatment at left separators)
    # l_aqueous: Int64 (Unit Cost of aqueous water water treatment disposal at right separators)
    # l_organic: Int64 (Unit Cost of organic waste water treatment disposal at right separators)
# r_matrix: 5x4 matrix
    # Rows: Reactor index
    # Columns: [1] Fixed Cost, [2] Flow Cost, [3] Capacity, [4] Conversion
# s_matrix: 5x4 matrix [Right Separators]
    # Rows: Separator (Right) index
    # Columns: [1] Fixed Cost, [2] Flow Cost, [3] Capacity, [4] Recovery
# k_matrix: 6x2 matrix
    # Rows: Customer index
    # Columns: [1] Price (USD/MT), [2] Demand, [3] Purity

## Ranges & Value Vectors
material_range = 1:5
small_material_range = 1:2

chemical_MW = p_matrix[:, 2]

full_supplier_range = 1:(size(n_p1_matrix)[1] + size(n_p2_matrix)[1])
p1_supplier_range = 1:size(n_p1_matrix)[1]
p2_supplier_range = (size(n_p1_matrix)[1] + 1):(size(n_p1_matrix)[1] + size(n_p2_matrix)[1])
supplier_unit_price = n_matrix[:, 1]
supplier_supply = n_matrix[:, 2]
supplier_impurity = n_matrix[:, 3]

left_separator_range = 1:size(m_matrix)[1]
left_separator_fixed_cost = m_matrix[:, 1]
left_separator_unit_cost = m_matrix[:, 2]
left_separator_capacity = m_matrix[:, 3]

left_waste_range = 1:1
left_waste_unit_cost = waste_matrix[:, 1]

right_waste_range = 1:1
right_waste_aqueous_unit_cost = waste_matrix[:, 2]
right_waste_organic_unit_cost = waste_matrix[:, 3]

reactor_range = 1:size(r_matrix)[1]
reactor_fixed_cost = r_matrix[:, 1]
reactor_unit_cost = r_matrix[:, 2]
reactor_capacity = r_matrix[:, 3]
reactor_conversion_pct = r_matrix[:, 4]

right_separator_range = 1:size(s_matrix)[1]
right_separator_fixed_cost = s_matrix[:, 1]
right_separator_unit_cost = s_matrix[:, 2]
right_separator_capacity = s_matrix[:, 3]
right_separator_recovery_pct = s_matrix[:, 4]

customer_range = 1:size(k_matrix)[1]
customer_unit_revenue = k_matrix[:, 1]
customer_demand = k_matrix[:, 2]
customer_purity_pct = k_matrix[:, 3];

big_M = 1e9

1.0e9

\begin{align}
    \text{Variables and Sign Restrictions:} \\
    a_{nm}^{p} \in \mathbb{R}_{+} =&\ \text{amount of material $p$ purchased from supplier $n$ and sent to separator $m$} \\
    I_{n} =&\ \text{\% impurity in material $p$ purchased from supplier $n$} \\ \\
    b_{mr}^{p} \in \mathbb{R}_{+} =&\ \text{amount of separated (pure) material $p$ to send from separator $m$ to reactor $r$} \\
    \Delta b_{mj}^{p} \in \mathbb{R}_{+} =&\ \text{amount of separated (impure) material $p$ to send from separator $m$ to waste plant $j$} \\ \\
    d_{rs}^{p} \in \mathbb{R}_{+} =&\ \text{amount of reacted material $p$ (with impurities) to ship from reactor $r$ to separator $s$} \\
    \gamma_{r} =&\ \text{\% yield during reaction from reactor $r$} \\ \\
    e_{sk}^{p} \in \mathbb{R}_{+} =&\ \text{amount of separated material $p$ (pure) to ship from separator $s$ to customer $k$} \\
    \Delta e_{sl}^{A} \in \mathbb{R}_{+} =&\ \text{amount of separated (impure) aqueous material to ship from separator $s$ to waste plant $l$} \\
    \Delta e_{sl}^{O} \in \mathbb{R}_{+} =&\ \text{amount of separated (impure) organic material to ship from separator $s$ to waste plant $l$} \\ 
    I_{s} =&\ \text{\% recovery of materials from separator $s$} \\ \\
    g_{k} \in \mathbb{R}_{+} =&\ \text{units of demand from customer k} \\ 
    \nu_{k} =&\ \text{maximum \% of non-ECH product that customer $k$ willing to except, relative to amount of ECH shipped}\\ \\
    \rho_{m} \in \{0, 1\} =&\ \text{binary indicator if separator (left) $m$ used} \\
    \sigma_{r} \in \{0, 1\} =&\ \text{binary indicator if reactor $r$ used} \\
    \mu_{s} \in \{0, 1\} =&\ \text{binary indicator if separator (right) $s$ used} \\
\end{align}

In [147]:
model = Model(Gurobi.Optimizer)

## CREATE VARIABLES
@variable(model,
    a[n in full_supplier_range, m in left_separator_range, p in small_material_range] >= 0)

@variable(model,
    b[m in left_separator_range, r in reactor_range, p in small_material_range] >= 0)
@variable(model,
    delta_b[m in left_separator_range, j in left_waste_range, p in small_material_range] >= 0)

@variable(model,
    d[r in reactor_range, s in right_separator_range, p in material_range] >= 0)

@variable(model,
    e[s in right_separator_range, k in customer_range, p in material_range] >= 0)
@variable(model,
    delta_e[s in right_separator_range, l in right_waste_range, p in material_range] >= 0)

@variable(model,
    rho[m in left_separator_range], Bin)
@variable(model,
    sigma[r in reactor_range], Bin)
@variable(model,
    mu[s in right_separator_range], Bin)

@variable(model,
    FC_m >= 0)
@variable(model,
    FC_r >= 0)
@variable(model,
    FC_s >= 0)

@variable(model,
    UC_n >= 0)
@variable(model,
    UC_m >= 0)
@variable(model,
    UC_r >= 0)
@variable(model,
    UC_s >= 0)
@variable(model,
    UC_j >= 0)
@variable(model,
    UC_l >= 0)

@variable(model,
    R_k >= 0)

## ONLY CERTAIN SUPPLIERS CAN PROVIDE CERTAIN PRODUCTS
@constraint(model,
    DCH_to_L_separator_flow[n in p1_supplier_range, m in left_separator_range],
    a[n, m, 2] == 0)  #a[1:5, 1:M, 2]
@constraint(model,
    NaOH_to_L_separator_flow[n in p2_supplier_range, m in left_separator_range],
    a[n, m, 1] == 0)  #a[6:10, 1:M, 1] 
# @constraint(model,
#     Other_to_L_separator_flow[n in full_supplier_range, m in left_separator_range, p in [3, 4, 5]],
#     a[n, m, p] == 0)  #a[1:10, 1:M, 3:5]

## ONLY CERTAIN PRODUCTS CAN BE SEPARATED
# @constraint(model,
#     L_separator_to_reactor_flow[m in left_separator_range, r in reactor_range, p in [3, 4, 5]],
#     b[m, r, p] == 0)
# @constraint(model,
#     L_separator_to_waste_flow[m in left_separator_range, r in reactor_range, p in [3, 4, 5]],
#     b[m, r, p] == 0);

## WHAT GOES INTO L SEPARATORS MUST COME OUT
@constraint(model,
    in_and_out_L_separator_flow[m in left_separator_range, p in small_material_range],
    sum( (a[n, m, p]) for n in full_supplier_range) == 
    sum( (delta_b[m, j, p]) for j in left_waste_range) + 
    sum( (b[m, r, p]) for r in reactor_range));

## WHAT GOES INTO REACTORS MUST COME OUT
@constraint(model,
    in_and_out_reactor_flow[r in reactor_range],
    sum( sum( (b[m, r, p]) for p in small_material_range) for m in left_separator_range) == 
    sum( sum( (d[r, s, p]) for p in material_range) for s in right_separator_range));

## WHAT GOES INTO R SEPARATORS MUST COME OUT
@constraint(model,
    in_and_out_R_separator_flow[s in right_separator_range, p in material_range],
    sum( (d[r, s, p]) for r in reactor_range) == 
    sum( (delta_e[s, l, p]) for l in right_waste_range) + 
    sum( (e[s, k, p]) for k in customer_range));

## ENFORCE FIXED-COST OF L SEPARATORS
@constraint(model,
    Fixed_Cost_L_Separators_1[n in full_supplier_range, m in left_separator_range, p in small_material_range],
    a[n, m, p] <= big_M * rho[m])
@constraint(model,
    Fixed_Cost_L_Separators_2[m in left_separator_range, r in reactor_range, p in small_material_range],
    b[m, r, p] <= big_M * rho[m])
@constraint(model,
    Fixed_Cost_L_Separators_3[m in left_separator_range, j in left_waste_range, p in small_material_range],
    delta_b[m, j, p] <= big_M * rho[m]);
@constraint(model,
    FC_m == sum( (left_separator_fixed_cost[m] * rho[m]) for m in left_separator_range));

## ENFORCE FIXED-COST OF REACTORS
@constraint(model,
    Fixed_Cost_Reactors_1[m in left_separator_range, r in reactor_range, p in small_material_range],
    b[m, r, p] <= big_M * sigma[r])
@constraint(model,
    Fixed_Cost_Reactors_2[r in reactor_range, s in right_separator_range, p in material_range],
    d[r, s, p] <= big_M * sigma[r])
@constraint(model,
    FC_r == sum( (reactor_fixed_cost[r] * sigma[r]) for r in reactor_range));

## ENFORCE FIXED-COST OF R SEPARATORS
@constraint(model,
    Fixed_Cost_R_Separators_1[r in reactor_range, s in right_separator_range, p in material_range],
    d[r, s, p] <= big_M * mu[s])
@constraint(model,
    Fixed_Cost_R_Separators_2[s in right_separator_range, k in customer_range, p in material_range],
    e[s, k, p] <= big_M * mu[s])
@constraint(model,
    Fixed_Cost_R_Separators_3[s in right_separator_range, l in right_waste_range, p in material_range],
    delta_e[s, l, p] <= big_M * mu[s]);
@constraint(model,
    FC_s == sum( (right_separator_fixed_cost[s] * mu[s]) for s in right_separator_range))

## CUSTOMER DEMAND REQUIREMENTS
@constraint(model,
    Customer_Demand[k in customer_range],
    sum( (e[s, k, 5]) for s in right_separator_range) <=
    customer_demand[k]);
@constraint(model,
    Customer_Product_Purity[k in customer_range],
    sum( (e[s, k, 1] + e[s, k, 2] + e[s, k, 3] + e[s, k, 4]) for s in right_separator_range)
    <= customer_purity_pct[k] * sum( (e[s, k, 5]) for s in right_separator_range))


## CALCULATE UNIT COSTS
@constraint(model,
    UC_n == sum( 
                sum( 
                    sum( 
                        (supplier_unit_price[n] * a[n, m, p]) 
                    for p in small_material_range) 
                for m in left_separator_range) 
            for n in full_supplier_range))
@constraint(model,
    UC_m == sum( 
                sum( 
                    sum( 
                        (left_separator_unit_cost[m] * a[n, m, p]) 
                    for p in small_material_range) 
                for m in left_separator_range) 
            for n in full_supplier_range))
@constraint(model,
    UC_j == sum( 
                sum( 
                    sum( 
                        (left_waste_unit_cost[j] * delta_b[m, j, p]) 
                    for p in small_material_range) 
                for j in left_waste_range) 
            for m in left_separator_range))
@constraint(model,
    UC_r == sum( 
                sum( 
                    sum( 
                        (reactor_unit_cost[r] * b[m, r, p]) 
                    for p in small_material_range) 
                for r in reactor_range) 
            for m in left_separator_range))
@constraint(model,
    UC_s == sum( 
                sum( 
                    sum( 
                        (right_separator_unit_cost[s] * d[r, s, p]) 
                    for p in material_range) 
                for s in right_separator_range) 
            for r in reactor_range));
@constraint(model,
    UC_l == sum( 
                sum( 
                    sum( 
                        (right_waste_aqueous_unit_cost[l] * delta_e[s, l, p]) 
                    for p in 2:4) 
                for l in right_waste_range) 
            for s in right_separator_range) + 
            sum( 
                sum( 
                    (right_waste_organic_unit_cost[l] * delta_e[s, l, 1]) 
                for l in right_waste_range) 
            for s in right_separator_range))

## SUPPLIERS HAVE LIMITED CAPACITY
@constraint(model,
    Limit_DCH_supplier_volume[n in p1_supplier_range],
    sum( (a[n, m, 1]) for m in left_separator_range) 
    <= supplier_supply[n])
@constraint(model,
    Limit_NaOH_supplier_volume[n in p2_supplier_range],
    sum( (a[n, m, 2]) for m in left_separator_range) 
    <= supplier_supply[n])

## L SEPARATORS HAVE LIMITED CAPACITY
@constraint(model,
    L_separator_capacity_in[m in left_separator_range],
    sum( sum( (a[n, m, p]) for p in small_material_range) for n in full_supplier_range) 
    <= left_separator_capacity[m])
@constraint(model,
    L_separator_capacity_out[m in left_separator_range],
    sum( sum( (b[m, r, p]) for p in small_material_range) for r in reactor_range) + 
    sum( sum( (delta_b[m, j, p]) for p in small_material_range) for j in left_waste_range) 
    <= left_separator_capacity[m])

## REACTORS HAVE LIMITED CAPACITY
@constraint(model,
    reactor_capacity_in[r in reactor_range],
    sum( sum( (b[m, r, p]) for p in small_material_range) for m in left_separator_range)
    <= reactor_capacity[r])

## R SEPARATORS HAVE LIMITED CAPACITY
@constraint(model,
    R_separator_capacity_in[s in right_separator_range],
    sum( sum( (d[r, s, p]) for p in material_range) for r in reactor_range) 
    <= right_separator_capacity[s])
@constraint(model,
    R_separator_capacity_out[s in right_separator_range],
    sum( sum( (e[s, k, p]) for p in material_range) for k in customer_range) + 
    sum( sum( (delta_e[s, l, p]) for p in material_range) for l in right_waste_range) 
    <= right_separator_capacity[s]);

## CALCULATIONS (SUPPLIERS -> L SEPARATORS)
@constraint(model,
    DCH_supplier_impurity_calc_1[m in left_separator_range],
    sum( ( (1 - supplier_impurity[n]) * (a[n, m, 1]) ) for n in p1_supplier_range) == 
    sum( (b[m, r, 1]) for r in reactor_range))
@constraint(model,
    NaOH_supplier_impurity_calc_1[m in left_separator_range],
    sum( ( (1 - supplier_impurity[n]) * (a[n, m, 2]) ) for n in p2_supplier_range) == 
    sum( (b[m, r, 2]) for r in reactor_range))
@constraint(model,
    DCH_supplier_impurity_calc_2[m in left_separator_range],
    sum( (supplier_impurity[n] * a[n, m, 1]) for n in p1_supplier_range) == 
    sum( (delta_b[m, j, 1]) for j in left_waste_range))
@constraint(model,
    NaOH_supplier_impurity_calc_2[m in left_separator_range],
    sum( (supplier_impurity[n] * a[n, m, 2]) for n in p2_supplier_range) == 
    sum( (delta_b[m, j, 2]) for j in left_waste_range))

## CALCULATIONS (L SEPARATORS -> REACTORS)
@constraint(model,
    DCH_reaction[r in reactor_range],
    sum( (d[r, s, 1]) for s in right_separator_range) == 
    sum( 
        ( chemical_MW[1] * ( (b[m, r, 1]/chemical_MW[1]) - ((reactor_conversion_pct[r]) * (b[m, r, 1]/chemical_MW[1])) ) ) 
    for m in left_separator_range))
@constraint(model,
    NaOH_reaction[r in reactor_range],
    sum( (d[r, s, 2]) for s in right_separator_range) == 
    sum( 
        ( chemical_MW[2] * ( (b[m, r, 2]/chemical_MW[2]) - ((reactor_conversion_pct[r]) * (b[m, r, 1]/chemical_MW[1])) ) ) 
    for m in left_separator_range))
@constraint(model,
    NaCl_reaction[r in reactor_range],
    sum( (d[r, s, 3]) for s in right_separator_range) == 
    sum( 
        ( chemical_MW[3] * reactor_conversion_pct[r] * (b[m, r, 1]/chemical_MW[1]) ) 
    for m in left_separator_range))
@constraint(model,
    H2O_reaction[r in reactor_range],
    sum( (d[r, s, 4]) for s in right_separator_range) == 
    sum( 
        ( chemical_MW[4] * reactor_conversion_pct[r] * (b[m, r, 1]/chemical_MW[1]) ) 
    for m in left_separator_range))
@constraint(model,
    ECH_reaction[r in reactor_range],
    sum( (d[r, s, 5]) for s in right_separator_range) == 
    sum( 
        ( chemical_MW[5] * reactor_conversion_pct[r] * (b[m, r, 1]/chemical_MW[1]) ) 
    for m in left_separator_range));

## CALCULATIONS (REACTORS -> R SEPARATORS)
@constraint(model,
    material_R_separation_to_waste[s in right_separator_range, p in 1:4],
    sum( (right_separator_recovery_pct[s] * d[r, s, p]) for r in reactor_range) == 
    sum( (delta_e[s, l, p]) for l in right_waste_range)) 
@constraint(model,
    ECH_r_separation[s in right_separator_range],
    sum( (d[r, s, 5]) for r in reactor_range) ==
    sum( (e[s, k, 5]) for k in customer_range))
@constraint(model,
    material_R_separation_to_customers[s in right_separator_range, p in 1:4],
    sum( ( (1 - right_separator_recovery_pct[s]) * (d[r, s, p]) ) for r in reactor_range) == 
    sum( (e[s, k, p]) for k in customer_range));

## REVENUE FROM CUSTOMER SALES
@constraint(model,
    R_k == sum( 
                sum( 
                    (customer_unit_revenue[k] * e[s, k, 5]) 
                for k in customer_range) 
            for s in right_separator_range))

## JUST BE BE SURE (LOL)
@constraint(model,
    right_waste_sanity_check[s in right_separator_range, l in right_waste_range],
    delta_e[s, l, 5] == 0)

# print(model)  #use to spot check

## OBJECTIVE FUNCTION (MAX NET PROFIT)
@objective(model,
    Max,
    R_k);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-18


In [148]:
optimize!(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 872 rows, 485 columns and 3815 nonzeros
Model fingerprint: 0x6d7bfb47
Variable types: 470 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [5e-03, 1e+09]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 8e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective -0.0000000
Presolve removed 719 rows and 104 columns
Presolve time: 0.00s
Presolved: 153 rows, 381 columns, 1683 nonzeros
Variable types: 381 continuous, 0 integer (0 binary)

Root relaxation: objective 5.095575e+07, 94 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

In [149]:
value.(a)  #supplier x left separator

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, 1:10
    Dimension 2, 1:5
    Dimension 3, 1:2
And data, a 10×5×2 Array{Float64, 3}:
[:, :, 1] =
    0.0   0.0  0.0  5000.0  5000.0
    0.0   0.0  0.0  5000.0  5000.0
 5000.0   0.0  0.0     0.0     0.0
    0.0   0.0  0.0  5000.0  5000.0
 2939.81  0.0  0.0  1030.1  1030.1
    0.0   0.0  0.0     0.0     0.0
    0.0   0.0  0.0     0.0     0.0
    0.0   0.0  0.0     0.0     0.0
    0.0   0.0  0.0     0.0     0.0
    0.0   0.0  0.0     0.0     0.0

[:, :, 2] =
 0.0  0.0     0.0     0.0       0.0
 0.0  0.0     0.0     0.0       0.0
 0.0  0.0     0.0     0.0       0.0
 0.0  0.0     0.0     0.0       0.0
 0.0  0.0     0.0     0.0       0.0
 0.0  0.0     0.0   655.803   655.803
 0.0  0.0     0.0  1000.0    1000.0
 0.0  0.0  3000.0     0.0       0.0
 0.0  0.0     0.0  1000.0    1000.0
 0.0  0.0     0.0  1250.0    1250.0

In [151]:
value.(delta_b)  #left separator x waste plant

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, 1:5
    Dimension 2, 1:1
    Dimension 3, 1:2
And data, a 5×1×2 Array{Float64, 3}:
[:, :, 1] =
   83.79615703023808
    0.0
    0.0
 1470.601921484881
 1470.601921484881

[:, :, 2] =
   0.0
   0.0
 210.00000000000003
  90.290156224881
  90.290156224881

In [162]:
value.(b)  #left separator x reactor

47396.02593654547

In [163]:
value.(d)  #reactor x right separator

47390.8154235772

In [153]:
value.(e)  #right separator x customers

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, 1:5
    Dimension 2, 1:6
    Dimension 3, 1:5
And data, a 5×6×5 Array{Float64, 3}:
[:, :, 1] =
   0.0      0.0  0.0  0.0  0.0  7.65097
  47.1361   0.0  0.0  0.0  0.0  0.0
   7.81792  0.0  0.0  0.0  0.0  0.0
   0.0      0.0  0.0  0.0  0.0  0.0
 188.565    0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
   0.0      0.0     0.0  0.0  629.109  0.0
   0.0      0.0     0.0  0.0    0.0    0.0
  21.1438   0.0     0.0  0.0    0.0    0.0
 100.733   46.7603  0.0  0.0  553.259  0.0
 257.756    0.0     0.0  0.0    0.0    0.0

[:, :, 4] =
   0.0    313.24  50.0  0.0  0.0  0.0
   0.0      0.0    0.0  0.0  0.0  0.0
   0.0      0.0    0.0  0.0  0.0  0.0
   0.0      0.0    0.0  0.0  0.0  0.0
 126.848    0.0    0.0  0.0  0.0  0.0

[:, :, 5] =
    0.0     0.0     0.0     0.0     

In [154]:
value.(delta_e)  #right separator x right waste plant

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, 1:5
    Dimension 2, 1:1
    Dimension 3, 1:5
And data, a 5×1×5 Array{Float64, 3}:
[:, :, 1] =
   68.85870209989332
  895.5853331709096
  773.9738150078792
    0.0
 1382.8135753874142

[:, :, 2] =
 0.0
 0.0
 0.0
 0.0
 0.0

[:, :, 3] =
 5661.984199103995
    0.0
 2093.2315236747345
 3970.9344208987586
 1890.2070072998458

[:, :, 4] =
 3269.157098796113
    0.0
    0.0
    0.0
  930.2171076046978

[:, :, 5] =
 0.0
 0.0
 0.0
 0.0
 0.0

In [155]:
value.(UC_n)  #unit cost of purchasing

2.780580312449762e7

In [156]:
value.(UC_m), value.(FC_m)  #unit cost of left separators, fixed cost of using left separators

(2.3518623284749286e6, 675000.0)

In [157]:
value.(UC_r), value.(FC_r)  #unit cost of reactors, fixed cost of using reactors

(6.191682074923639e6, 3.0e6)

In [158]:
value.(UC_s), value.(FC_s)  #unit cost of right separators, fixed cost of using right separators

(2.3695407711788598e6, 675000.0)

In [159]:
value.(UC_j)  #unit cost of left waste plant disposal

341558.0312449762

In [160]:
value.(UC_l)  #unit cost of right waste plant disposal

3.342188848570863e6

In [161]:
value.(R_k)  #revenue of selling to customers

5.0955749486844525e7

\begin{align}
    \text{Flow Constraints (MT units):}& \\
        &\sum_{n=1}^{N} a_{nm}^{p} = \sum_{j=1}^{J} \Delta b_{mj}^{p} + \sum_{r=1}^{R} b^{p}_{mr}\ , \quad \forall p \in \{1, 2\}, \forall m \in M, \forall j \in J \\
        &\sum_{m=1}^{M} \sum_{p=1}^{P} b_{mr}^{p} = \sum_{s=1}^{S}d_{rs}^{p}\ , \quad \forall r \in R, \forall p \in \{1, 2\} \\
        &\sum_{r=1}^{R} d_{rs}^{p} = \sum_{l=1}^{L} \Delta e_{sl}^{p} + \sum_{k=1}^{K} e_{sk}^{p}\ , \quad \forall s \in S, \forall p \in P \\
        &a_{nm}^{p} = b_{mr}^{p} = \Delta b_{mj}^{p} = 0\ , \quad \forall p \in \{3, 4, 5\}, \forall n \in N, \forall m \in M \\ \\ 
    \text{Fixed Cost for Left Separators:}& \\
        &a_{nm}^{p} \leq \mathbf{M} \times \rho_{m}\ , \quad \forall n \in N, \forall p \in P, \forall m \in M \\
        &b_{mr}^{p} \leq \mathbf{M} \times \rho_{m}\ , \quad \forall p \in P, \forall m \in M, \forall r \in R \\
        &\Delta b_{mj}^{p} \leq \mathbf{M} \times \rho_{m}\ , \quad \forall p \in P, \forall m \in M, \forall j \in J \\
        &FC_{m} = \sum_{m=1}^{M} f_{m} \times \rho_{m} \\
    \text{Fixed Cost for Reactors:}& \\
        &b_{mr}^{p} \leq \mathbf{M} \times \sigma_{r}\ , \quad \forall p \in P, \forall m \in M, \forall r \in R \\
        &d_{rs}^{p} \leq \mathbf{M} \times \sigma_{r}\ , \quad \forall r \in R, \forall p \in P, \forall s \in S \\
        &FC_{r} = \sum_{r=1}^{R} f_{r} \times \sigma_{r} \\ \\
    \text{Fixed Cost for Right Separators:}& \\
        &d_{rs}^{p} \leq \mathbf{M} \times \mu_{s}\ , \quad \forall r \in R, \forall p \in P, \forall s \in S \\
        &e_{sk}^{p} \leq \mathbf{M} \times \mu_{s}\ , \quad \forall s \in S, \forall p \in P, \forall k \in K \\
        &\Delta e_{sl}^{p} \leq \mathbf{M} \times \mu_{s}\ , \quad \forall s \in S, \forall p \in P, \forall l \in L \\
        &FC_{s} = \sum_{s=1}^{S} f_{r} \times \mu_{s} \\ \\
    \text{Demand Constraints:}& \\
        &\sum_{s=1}^{S} e_{sk}^{5} \leq g_{k}\ , \quad \forall k \in K \\ 
        &\sum_{s=1}^{S} \sum_{p=1}^{4} e_{sk}^{p} \leq \nu_{k} \times \sum_{s=1}^{S} e_{sk}^{5}\ , \quad \forall k \in K\\
    \text{Calculate Unit Costs:}& \\
        &UC_{n} = \sum_{p=1}^{2} \sum_{n=1}^{N} \sum_{m=1}^{M} c_{n} \times a_{nm}^{p} \\
        &UC_{m} = \sum_{p=1}^{P} \sum_{n=1}^{N} \sum_{m=1}^{M} c_{m} \times a_{nm}^{p} \\
        &UC_{j} = \sum_{p=1}^{P} \sum_{m=1}^{M} \sum_{j=1}^{J} c_{j} \times \Delta b_{mj}^{p} \\
        &UC_{r} = \sum_{p=1}^{P} \sum_{m=1}^{M}\sum_{r=1}^{R} c_{r} \times b_{mr}^{p} \\
        &UC_{s} = \sum_{p=1}^{P} \sum_{r=1}^{R} \sum_{s=1}^{S} c_{s} \times d_{rs}^{p} \\
        &UC_{l}= \sum_{s=1}^{S} \sum_{l=1}^{L} (c_{l}^{A} \times \Delta e_{sl}^{A} + c_{l}^{O} \times \Delta e_{sl}^{O}) \\ \\
    \text{Capacity Constriants:}& \\
        &\sum_{m=1}^{M} a_{nm}^{p} \leq supply_{N}\ , \quad \forall n \in N, \forall p \in \{1, 2\} \\
        &\sum_{n=1}^{N} \sum_{p=1}^{2} a_{nm}^{p} \leq capacity_{M}\ , \quad \forall m \in M \\
        &\sum_{j=1}^{J} \Delta b_{mj}^{p} + \sum_{r=1}^{R} \sum_{p=1}^{2} b_{mr}^{p} \leq capacity_{M}\ , \quad \forall m \in M \\
        &\sum_{m=1}^{M} \sum_{p=1}^{2} b_{mr}^{p} \leq capacity_{R}\ , \quad \forall r \in R \\
        &\sum_{s=1}^{S} \sum_{p=1}^{2} d_{rs}^{p} \leq capacity_{R}\ , \quad \forall r \in R \\
        &\sum_{r=1}^{R} \sum_{p=1}^{P} d_{rs}^{p} \leq capacity_{S}\ , \quad \forall s \in S \\
        &\sum_{l=1}^{L} \sum_{p=1}^{P} \Delta e_{sl}^{p} + \sum_{p=1}^{P} \sum_{k=1}^{K} e_{sk}^{p} \leq capacity_{S}\ , \quad \forall s \in S \\
\end{align}

\begin{align}
    \text{Calculations (Suppliers $\rightarrow$ Left Separators):}& \\
        &\sum_{n=1}^{N} (1 - I_{n}) \times a_{nm}^{p} = \sum_{r=1}^{R} b_{mr}^{p}\ , \quad \forall m \in M, \forall p \in \{1, 2\} \\
        &\sum_{n=1}^{N} I_{n} \times a_{nm}^{p} = \sum_{j=1}^{J} \Delta b_{mj}^{p}\ , \quad \forall m \in M, \forall p \in \{1, 2\} \\ \\
    \text{Calculations (Left Separators $\rightarrow$ Reactors):}& \\
        &\sum_{m=1}^{M} MW_{p=1} \times (\frac{b_{mr}^{p=1}}{MW_{p=1}} - \gamma_{r} \times \frac{b_{mr}^{p=1}}{MW_{p=1}} ) = \sum_{s=1}^{S} d_{rs}^{p=1}\ , \quad \forall r \in R \\
        &\sum_{m=1}^{M} MW_{p=2} \times (\frac{b_{mr}^{p=2}}{MW_{p=2}} - \gamma_{r} \times \frac{b_{mr}^{p=1}}{MW_{p=1}} ) = \sum_{s=1}^{S} d_{rs}^{p=2}\ , \quad \forall r \in R \\
        &\sum_{m=1}^{M} (MW_{p=3} \times \gamma_{r} \times \frac{b_{mr}^{p=1}}{MW_{p=1}} ) = \sum_{s=1}^{S} d_{rs}^{p=3}\ , \quad \forall r \in R \\
        &\sum_{m=1}^{M} (MW_{p=4} \times \gamma_{r} \times \frac{b_{mr}^{p=1}}{MW_{p=1}} ) = \sum_{s=1}^{S} d_{rs}^{p=4}\ , \quad \forall r \in R \\
        &\sum_{m=1}^{M} (MW_{p=5} \times \gamma_{r} \times \frac{b_{mr}^{p=1}}{MW_{p=1}} ) = \sum_{s=1}^{S} d_{rs}^{p=5}\ , \quad \forall r \in R \\ \\
    \text{Calculations (Reactors $\rightarrow$ Right Separators):}& \\
        &\sum_{r=1}^{R} I_{s} \times d_{rs}^{p} = \sum_{l=1}^{L} \Delta e_{sl}^{p}\ , \quad \forall s \in S, \forall p \in \{1, 2, 3, 4\} \\
        &\sum_{r=1}^{R} d_{rs}^{5} = \sum_{k=1}^{K} e_{sk}^{5}\ , \quad \forall s \in S \\
        &\sum_{r=1}^{R} (1 - I_{s}) \times d_{rs}^{p} = \sum_{k=1}^{K} e_{sk}^{p}\ , \quad \forall s \in S, \forall p \in \{1, 2, 3, 4\}
\end{align}

\begin{align}
    & \text{Objective Function:}& \\
    \max & \quad R_{k} - (FC_{m} + UC_{m}) - (FC_{r} + UC_{r}) - (FC_{s} + UC_{s}) - UC_{n} - UC_{j} - UC_{l}
\end{align}